# inverse Compton scattering

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# To avoid copying things to GPU memory,
# ideally allocate everything in torch on the GPU
# and avoid non-torch function calls
import torch
torch.set_printoptions(precision=8) # Set displayed output precision to 10 digits
# Use this to enable GPU support and set the floating point precision

from torchquad import set_up_backend  # Necessary to enable GPU support
set_up_backend(backend="torch", data_type="float32")
from torchquad import Trapezoid, Simpson, Boole, GaussLegendre, MonteCarlo, VEGAS # The available integrators

01:45:02|TQ-WARNING| Error enabling CUDA. cuda.is_available() returned False. CPU will be used.


In [24]:
kB = 8.61733e-8 # keV / K
me_c2 = 510.99892 # keV
h = 4.13567e-18 # keV s

In [25]:
def Ne(gamma, Te, gamma_th, Ce=1, n=3):
    Te = torch.tensor([Te])
    gamma_th = torch.tensor(gamma_th)
    #gamma = torch.tensor(gamma)
    Ce = torch.tensor(Ce)
    n = torch.tensor(n)

    Theta = kB * Te / me_c2

    Ne_tensor = torch.zeros_like(gamma)

    for i, i_gamma in enumerate(gamma):
        Ne_tensor[i] = Ce*i_gamma**2*torch.exp(-i_gamma/Theta)/(2.*Theta**3) if i_gamma <= gamma_th else Ce*gamma_th**2*torch.exp(-gamma_th/Theta)*(i_gamma/gamma_th)**-n/(2.*Theta**3)

    return Ne_tensor

def beta(gamma):
    return torch.sqrt(1 - 1/gamma**2)

def e1(nu,theta,theta2,gamma,phi2):

    #nu = torch.tensor(nu)
    theta = torch.tensor(theta)
    #theta2 = torch.tensor(theta2)
    #gamma = torch.tensor(gamma)
    #phi2 = torch.tensor(phi2)
    
    e0 = h*nu
    cos_theta1 = torch.cos(theta)*torch.cos(theta2)+torch.sin(theta)*torch.sin(theta2)*torch.sin(phi2)
    
    return e0*(1-beta(gamma)*torch.cos(theta2))/(e0/(gamma*me_c2)*(1-torch.cos(theta))+1-beta(gamma)*cos_theta1)

In [26]:
gamma = torch.logspace(0, 2, 30)
nu = torch.logspace(13, 16, 30)

In [27]:
def integrand(x,Te,gamma_th,nu,theta):
    # gamma x[:,0]
    # theta2 x[:,1]
    # phi2 x[:,2]
    return Ne(x[:,0], Te, gamma_th)*e1(nu,theta,x[:,1],x[:,0],x[:,2])*torch.sin(x[:,1])

def integrand2(x,Te,gamma_th):
    return Ne(x[:,0], Te, gamma_th)

# The integration domain, dimensionality and number of evaluations
# For the calculate_grid method we need a Tensor and not a list.
integration_domain = torch.Tensor([[1., gamma.max()], 
                                   [0., np.pi], 
                                   [0., 2*np.pi]])
dim = 3
N = 1024

# Initialize the integrator, Trapezoid, Simpson, Boole, GaussLegendre
integrator = GaussLegendre()
# Calculate sample points and grid information for the result calculation
grid_points, hs, n_per_dim = integrator.calculate_grid(N, integration_domain)

In [32]:
Te=widgets.FloatSlider(description='Te',min=1e8, max=1e11, step=1e3, value=1e10)
gamma_th=widgets.FloatSlider(description='gamma_th',min=1, max=gamma.max(), step=1, value=20)
theta=widgets.FloatSlider(description='theta',min=0, max=np.pi, step=np.pi/16, value=np.pi/4)

ui = widgets.VBox([Te, gamma_th, theta])

def calc_img(Te, gamma_th, theta):

    fig = plt.figure(dpi=100,figsize=(15,5))
    ax1 = fig.add_subplot(121)

    Ne_tensor = Ne(gamma, Te=Te, gamma_th=gamma_th)
    ax1.plot(gamma, Ne_tensor,label=rf'$T_\mathrm{{e}} = 10^{{{np.log10(Te):.2f}}}$ K')

    ax1.axvline(gamma_th, color='k', linestyle='--', label=rf'$\gamma_\mathrm{{th}} = {gamma_th:.1f}$')

    ax1.set_xlabel(r'$\gamma$')
    ax1.set_ylabel(r'$N_\mathrm{e}/C_\mathrm{e}$')
    ax1.grid(which='major', axis='both')
    ax1.set_yscale('log')
    ax1.set_xscale('log')
    ax1.legend()

    ax2 = fig.add_subplot(122)

    integral2 = integrator.integrate(lambda x:Ne(x, Te=Te, gamma_th=gamma_th, Ce=1, n=3), dim=1, N=N, integration_domain=[[1,100]])

    final_value = []
    for i_nu in nu:
        # function_values, _ = integrator.evaluate_integrand(lambda x: integrand(x,Te=Te,gamma_th=gamma_th,nu=i_nu,theta=theta), grid_points)
        # integral1 = integrator.calculate_result(function_values, dim, n_per_dim, hs, integration_domain)
        integral1 = integrator.integrate(lambda x: integrand(x,Te=Te,gamma_th=gamma_th,nu=i_nu,theta=theta), dim=dim, N=N, integration_domain=integration_domain)

        final_value.append(integral1/(4*np.pi*integral2))

    ax2.plot(nu, final_value,label=fr'$T_\mathrm{{e}} = 10^{{{np.log10(Te):.2f}}} K, \gamma_\mathrm{{th}} = {gamma_th:.1f}, \theta = {theta:.1f}\,\mathrm{{rad}}$')

    ax2.set_xlabel(r'$\nu\,\mathrm{[Hz]}$')
    ax2.set_ylabel(r'$<e1>\,\mathrm{[keV]}$')
    ax2.grid(which='both', axis='both')
    ax2.set_yscale('log')
    ax2.set_xscale('log')
    ax2.legend()
    
out = widgets.interactive_output(calc_img, {'Te': Te, 'gamma_th': gamma_th, 'theta':theta})
display(ui, out)

Output()

In [36]:
integral1 = integrator.integrate(lambda x: integrand(x,Te=1e10,gamma_th=20,nu=1e15,theta=0), dim=dim, N=N, integration_domain=integration_domain)

integral2 = integrator.integrate(lambda x:Ne(x, Te=1e10, gamma_th=20, Ce=1, n=3), dim=1, N=N, integration_domain=[[1,100]])

integral1/(4*np.pi*integral2)

tensor(0.00405084, dtype=torch.float64)